Ile de France

In [1]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
url ='https://www.notaires.fr/fr/annuaires-notaire/recherche/localisation/paris' #ile de France
req =requests.get(url)
soup = bs(req.text,'html.parser')
Notary__Result__Name = soup.find_all('h2',{"class": 'Notary__Result__Name'})
Notary__Result__Office = soup.find_all('div',{"class": 'Notary__Result__Office'})
Notary__Result__Location = soup.find_all('span',{"class": 'Notary__Result__Location'})
Notary__Result__Location__Address = soup.find_all('span',{"class": 'Notary__Result__Location__Address'})
Notary__Result__Location__City = soup.find_all('span',{"class": 'Notary__Result__Location__City'})


KeyboardInterrupt: ignored

In [ ]:
req

In [ ]:
#Notary__Result__Name

In [ ]:

#listes de valeus
l1,l2,l3,l4,l5,l6,l7,l8,l9 = [],[],[],[],[],[],[],[],[]


site web notaire   & nom prenom

In [ ]:

for elt in Notary__Result__Name:
    #print("Notary__Result__Name: ",elt.a.text)
     try:
        #print(elt.a.text)
        l1.append(elt.a.text)
        l2.append("https://www.notaires.fr"+elt.a.attrs['href'])
     except:
            print("An exception occurred")
    #print(elt.a.attrs['href'])
#l2

In [ ]:
#l6

Address

In [ ]:
#Address
Notary__Result__Location__Address  = soup.find_all('span', class_ = 'Notary__Result__Location__Address')
for elt in Notary__Result__Location__Address:
    l3.append(elt.text)
#l3

Ville

In [ ]:
#Ville
Notary__Result__Location__City  = soup.find_all('span', class_ = 'Notary__Result__Location__City')
for elt in Notary__Result__Location__City:
    #print("Notary__Result__Location__City: ", elt.text)
        l4.append(elt.text)


Nom notaire

In [ ]:

#Nom notaire
for elt in Notary__Result__Office:
    nom_notaire = bs(str(elt)).get_text()
    l5.append(nom_notaire)

CONVERTIR LES LISTES EN DATAFRAME-------------


In [ ]:

#------------CONVERTIR LES LISTES EN DATAFRAME-------------

#df1.columns =['Nom et prenom']  
df1= pd.DataFrame(l1)
df1.columns = ['Nom et prenom']
#separer nom et prenom
new = df1["Nom et prenom"].str.split(" ", n = 1, expand = True)
# making separate first name column from new data frame
df1["Prenom"]= new[0]
# making separate last name column from new data frame
df1["Nom"]= new[1]
df1.drop(df1.columns[[0]], axis = 1, inplace = True)

df2 = pd.DataFrame(l2)
df2.columns =['site web'] 
#df2.drop(df2.columns[[0]], axis = 1, inplace = True) #drop

df3= pd.DataFrame(l3)
df3.columns =['Adresse']

df4= pd.DataFrame(l4)
df4.columns = ['Ville']

df5= pd.DataFrame(l5)
df5.columns = ['Nom notaire']
#df5.drop(df5.columns[[0]], axis = 1, inplace = True) #drop

In [ ]:
df5

,Nom notaire
0,\n\n DUBAIL et associée
1,\n\n ALLEZ & ASSOCIES
2,\n\n Screeb Notaires
3,\n\n SARL JOURDE et associés
4,\n\n HUMBERT et associés
...,...
2050,\n\n HAUTEFEUILLE-HUARD et associée
2051,\n\n SAS GMH NOTAIRES
2052,"\n\n LAVAUD, notaire associé, SELARL"
2053,\n\n TURPAULT Frédérique


get adresse mail

In [ ]:
#------------------------------get adresse mail----------------------------------------------
#convert a site web column i the a list of url
list_url = df2["site web"].values.tolist()
url =list_url # url égale à la liste des adresse  web

#parcourir la liste
for i in range(len(list_url)):
    req1 =requests.get(list_url[i]) #faire une requettte sr chacune de site web
    soup1 = bs(req1.text,'html.parser')
    Notary__Section__Link = soup1.find_all('span',{"class": 'Notary__Section__Link'})[2].find_all(href=True)
    l6.append(Notary__Section__Link)

df6= pd.DataFrame(l6) #site
df6.columns =['Adresse email'] 


df4["Code Postal"] = df4['Ville'].str.extract(r'(\d+)').astype(str) #extraire code postal
df4["Ville"] = df4['Ville'].str.replace('\d+', '') #remove int in the ville


Concatener les daataframes

In [ ]:
#Concatener les daataframes
df= pd.concat([df1,df2,df3,df4,df5, df6], axis=1)


Save dataframe

In [ ]:
ile_de_france = df
ile_de_france.to_excel('ile_de_france.xlsx')
df = pd.read_excel (r'/content/ile_de_france.xlsx')
#df=df[['Adresse email']]
df['Adresse email'] = df['Adresse email'].str[16:] #  remove 16 premieres
df['Adresse email'] = df['Adresse email'].str[:1 -34]
df

,Unnamed: 0,Prenom,Nom,site web,Adresse,Ville,Code Postal,Nom notaire,Adresse email
0,0,Anne,BRAZILIER-VILLARD,https://www.notaires.fr/fr/notaire/anne-brazil...,53 AVENUE VICTOR HUGO,PARIS,75016,\n\n DUBAIL et associée,scp53victorhugo@paris.notaires.fr
1,1,Alexandre,OLLAGNON,https://www.notaires.fr/fr/notaire/alexandre-o...,5 rue Alfred de Vigny,PARIS,75008,\n\n ALLEZ & ASSOCIES,allezetassocies@allezetassocies.com
2,2,Damien,MANCEL,https://www.notaires.fr/fr/notaire/damien-manc...,64 AVENUE KLEBER,PARIS,75116,\n\n Screeb Notaires,accueil.uva@paris.notaires.fr
3,3,Jean-Michel,ROUVRAIS,https://www.notaires.fr/fr/notaire/jean-michel...,13 RUE AUBER,PARIS,75009,\n\n SARL JOURDE et associés,sjjr@paris.notaires.fr
4,4,Alexis,BAUDRY,https://www.notaires.fr/fr/notaire/alexis-baud...,137 139 RUE MARCADET,PARIS,75018,\n\n HUMBERT et associés,notaires.montmartre@paris.notaires.fr
...,...,...,...,...,...,...,...,...,...
2050,2050,Josette,GAMEIRO,https://www.notaires.fr/fr/notaire/josette-gam...,1 AVENUE WALWEIN,MONTREUIL,93100,\n\n HAUTEFEUILLE-HUARD et associée,hbmontreuil@paris.notaires.fr
2051,2051,François,BURNEAU,https://www.notaires.fr/fr/notaire/francois-bu...,6 RUE ANDRE CHENIER,ISSY-LES-MOULINEAUX,92130,\n\n SAS GMH NOTAIRES,gmh@paris.notaires.fr
2052,2052,Philippe,LAVAUD,https://www.notaires.fr/fr/notaire/philippe-la...,2 RUE MAURICE HARTMANN,ISSY-LES-MOULINEAUX,92130,"\n\n LAVAUD, notaire associé, SELARL",philippe.lavaud@paris.notaires.fr
2053,2053,Stéphanie,BOYER,https://www.notaires.fr/fr/notaire/stephanie-b...,219 AVENUE JEAN JAURES,BOBIGNY,93000,\n\n TURPAULT Frédérique,etude.turpault@paris.notaires.fr
